<a href="https://colab.research.google.com/github/anandchauhan21/pomegranate_disease_prediction/blob/main/Tcp_spi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gathering Database

In [ ]:
!pip install earthengine-api

In [ ]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=cIKO9asRxPYrzy88JdGcEmFoJsd3-e5ymQmqtfrKDfk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjy3-9H8XyR_SnBwaMZO5OzXfVDCojltYlaBv531yonFFDwitmLP_c

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import the Earth Engine API and initialize it.
import ee
ee.Initialize()

*** Earth Engine *** Python client initialized


In [ ]:
import folium

Peru = ee.Image("USGS/SRTMGL1_003")\
         .clip(ee.Geometry.Rectangle(-85, -20, -65,0))

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# Use folium to visualize the imagery.
mapid = Peru.getMapId({'pallete': ['black', 'white'], 'min': 0, 'max': 3000})

map = folium.Map(location=[-10, -75],zoom_start=5)
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map


In [ ]:
!pip install geetools

     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 44 kB 2.6 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=545e2412a4a175451235d2057ff2f35da078587c99cb1b752eb02db741f96cf2
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools


In [ ]:
import geetools

StartDate= ee.Date.fromYMD(1986,6,1)
EndDate = ee.Date.fromYMD(1986,12,31)

Area = ee.Geometry.Polygon(
    [[[43.86314427997206, 39.88060670550295],
      [43.86314427997206, 24.732304311990166],
      [63.55064427997206, 24.732304311990166],
      [63.55064427997206, 39.88060670550295]]])

collection =(ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5')
          .filterDate(StartDate,EndDate)
          .filterBounds(Area)
          .select('BT_CH4')
          .mean())


data_type = 'int16'
name_pattern = '{system_date}'
date_pattern = 'yMM' # dd: day, MMM: month (JAN), y: year

hansenImage = ee.Image("UMD/hansen/global_forest_change_2015").clip(Area)
visParams = {min:0,max: 1}
datamask = hansenImage.select('datamask')
mask = datamask.eq(1)
#print(mask)
print(collection)

ee.Image({
  "functionInvocationValue": {
    "functionName": "reduce.mean",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.select",
                    "arguments": {
                      "bandSelectors": {
                        "constantValue": [
                          "BT_CH4"
                        ]
                      },
                      "input": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      }
                    }
                  }
                }
              }
            },
            "collection": {
              "functionInvocationValue": 

In [ ]:
task = ee.batch.Export.image.toDrive(image=collection,
                                     description='hiiiiii',
                                     scale=1000,
                                     region=Area,
                                     fileNamePrefix='rockhh',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()


In [ ]:
task.status()

{'creation_timestamp_ms': 1649847620632,
 'description': 'hiiiiii',
 'id': 'AQCRHS6N4G3ZAGTRKDDRWRU2',
 'name': 'projects/earthengine-legacy/operations/AQCRHS6N4G3ZAGTRKDDRWRU2',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1649847620632}

In [ ]:
x = task.status()
#print(x)
print(x.get("state"));
#state = x.keys(state)
##stat2=task.status.state()
#print(stat2)
#while(x.get('state')!='COMPLETED'):
 # print(x.get("state"));

READY


In [ ]:

Export.image.toDrive(
  image: image1,#// define image
  description: 'TB198201',
  scale: 1000,//define scale meters/pixel
  region: roi

SyntaxError: ignored

In [ ]:
'''# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': mask,
    'description': 'maskIran',
    'folder':'GGG',
    'scale': 100,
    'region': Area
})
task.start()
print('ggg')'''

In [ ]:
folder_name = 'GEE_fol444'
tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=collection,
            folder=folder_name ,
            region=Area ,
            namePattern=name_pattern,
            scale=1000,
            dataType=data_type,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10
        )

AttributeError: ignored

# my try


In [ ]:
from logging import Filter
year = ee.List.sequence(1965,2010)
month = ee.List.sequence(1,12)
print(year)

def year_month_sum(year):
  def month_sum(month):
    w = collection.filter(ee.Filter.calendarRange(year,year,'year')).filter(ee.Filter.calendarRange(month,month,'month')).sum()
    return(w.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1)))
  return(month.map(month_sum))

print(year_month_sum(year))

ee.List({
  "functionInvocationValue": {
    "functionName": "List.sequence",
    "arguments": {
      "end": {
        "constantValue": 2010
      },
      "start": {
        "constantValue": 1965
      }
    }
  }
})


AttributeError: ignored

In [ ]:
monthlyPrecip = ee.ImageCollection.fromImages(year.map((year_month_sum(year))).flatten())

In [ ]:
def mnonthly_mean(m):
  w = monthlyPrecip.filter(ee.Filter.eq('month', m)).mean()
  
  return(w.set('month', m).
           set('system:time_start', ee.Date.fromYMD(1, m, 1)))

In [ ]:
meanMonthlyPrecipitation = ee.ImageCollection.fromImages(
  month.map(monthly_mean).flatten()
)

NameError: ignored

In [ ]:
print(meanMonthlyPrecipitation)

In [ ]:
print(year)

ee.List({
  "functionInvocationValue": {
    "functionName": "List.sequence",
    "arguments": {
      "end": {
        "constantValue": 2010
      },
      "start": {
        "constantValue": 1965
      }
    }
  }
})


In [ ]:
year = ee.List.sequence(1965,2010)
month = ee.List.sequence(1,12)

In [ ]:
import time

In [ ]:
for i in range(1965,2011):
  for j in range(1,12):
    for k in range(1,29):
      date_st = ee.Date.fromYMD(i,j,1)
      date_ed = ee.Date.fromYMD(i,j,k)
      
    collection =(ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5'))
    image2 = collection.filterDate(date_st,date_ed).filterBounds(Area).select('BT_CH4').mean()
    print(image2)
  
    #collection.mean().visualize(min=0.5)
    if j<10:
      FileName = 'TB'+str(i)+'0'+str(j)
    else:
      FileName = 'TB'+str(i)+str(j)
    data_type = 'int16'
    name_pattern = '{system_date}'
    date_pattern = 'yMM' # dd: day, MMM: month (JAN), y: year

    print(FileName)
    #hansenImage = ee.Image("UMD/hansen/global_forest_change_2015").clip(Area)
    #visParams = {min:0,max: 1}
    #datamask = hansenImage.select('datamask')
    #mask = datamask.eq(1)
    ####print(mask)

    #export function
    # run task
    x = task.status()
    while(x.get("state")=='RUNNING'):
      time.sleep(5)


    folder_name = 'GEE_fol44456'
    tasks = geetools.batch.Export.imagecollection.toDrive(
              collection=collection,
              folder=folder_name ,
              region=Area ,
              namePattern=name_pattern,
              scale=1000,
              dataType=data_type,
              datePattern=date_pattern,
              verbose=True,
              maxPixels=1e10
          )

    

NameError: ignored

In [ ]:
folder_name = 'GEE_fol4445'
tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=collection,
            folder=folder_name ,
            region=Area ,
            namePattern=name_pattern,
            scale=1000,
            dataType=data_type,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10
        )

AttributeError: ignored

# try2 

In [ ]:
import ee
ee.Initialize()

def scaleAndMask(img):
  return (img.select("LST_Day_1km")
          .multiply(0.02)
          .updateMask(img.select("QC_Day").eq(0))
          .copyProperties(img, ["system:time_start"]))

def makeMonthlyComposite(date):
  date = ee.Date(date)
  return (collection
          .filterDate(date, date.advance(1, 'month'))
          .mean()
          .set("system:index", date.format("YYYY-MM")))

collection = ee.ImageCollection("MODIS/006/MOD11A2").map(scaleAndMask)

start = ee.Date('2016-01-01')
end = ee.Date('2016-02-01')
n_months = end.difference(start, 'month').subtract(1)
months = ee.List.sequence(0, n_months).map(lambda n : start.advance(n, 'month'))
result = ee.ImageCollection(months.map(makeMonthlyComposite))
name="fill"
ids = result.aggregate_array("system:index").getInfo()
for id in ids:
  image = result.filter(ee.Filter.eq("system:index", id)).first()
  task = ee.batch.Export.image.toDrive(
      image=image,
      folder=folder_name ,
      region=Area ,
      namePattern=name_pattern,
      scale=1000,
      fileNamePrefix=name  
  )
  

NameError: ignored

In [ ]:
task.start()

# try section


In [ ]:
import ee
from dateutil.parser import parse

ee.Initialize()

# get Japan geometory as FeatureCollection from fusion table
japan = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw').filter(ee.Filter.eq('Country', 'Japan'))

In [ ]:
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').filter(ee.Filter.date('2014-01-01','2019-01-01')).select('avg_rad')

In [ ]:
dataset.mean().visualize(min=0.5).getDownloadURL(dict(name='thumbnail',region=[[[120.3345348936478, 46.853488838010854],[119.8071911436478, 24.598157870729043],[148.6353161436478, 24.75788466523463],[149.3384411436478, 46.61252884462868]]],scale=5000))

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5d708af401f83d42eed087f7b05ece27-07c6cf1a76ae69d9a41ead02800b200a:getPixels'

In [ ]:
# initialize output box
time0 = dataset.first().get('system:time_start');
first = dataset.first().reduceRegions(reducer=ee.Reducer.sum(),collection=japan,scale=1000).set('time_start', time0)

In [ ]:
# define reduceRegions function for iteration
def myfunc(image,first):
  added = image.reduceRegions(reducer=ee.Reducer.sum(),collection=japan,scale=1000).set('time_start', image.get('system:time_start'))
  return ee.FeatureCollection(first).merge(added)

# implement iteration
nightjp = dataset.filter(ee.Filter.date('2014-02-01','2019-01-01')).iterate(myfunc,first)

# get url to download
ee.FeatureCollection(nightjp).getDownloadURL(filetype='csv',selectors=ee.FeatureCollection(nightjp).first().propertyNames().getInfo())

EEException: ignored

In [ ]:
!pip install geemap

     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 5.4 MB 20.9 MB/s 
     |████████████████████████████████| 95 kB 3.3 MB/s 
     |████████████████████████████████| 8.5 MB 26.5 MB/s 
     |████████████████████████████████| 176 kB 49.0 MB/s 
     |████████████████████████████████| 130 kB 36.8 MB/s 
     |████████████████████████████████| 1.7 MB 44.3 MB/s 
     |████████████████████████████████| 1.3 MB 58.9 MB/s 
     |████████████████████████████████| 3.3 MB 29.0 MB/s 
     |████████████████████████████████| 98 kB 6.6 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 99 kB 7.5 MB/s 
     |████████████████████████████████| 1.2 MB 52.7 MB/s 
     |████████████████████████████████| 216 kB 53.5 MB/s 
     |████████████████████████████████| 97 kB 5.3 MB/s 
     |████████████████████████████████| 428 kB 55.5 MB/s 
     |████████████████████████████████| 397 kB 40.6 MB/s 
     |███████████████████

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5')
    .filterBounds(point)
    .filterDate('2016-01-01', '2016-12-31')
    .sort('CLOUD_COVER')
    .first()
    
)
surfaceReflectance = dataset.select('SREFL_CH1')


vis_params = {'min': -1000, 'max': 9000, 'palette': ['003b02', '006a03', '008d05', '01be07', '01ff09', 'ffffff']}

Map.centerObject(point, 8)
Map.addLayer(surfaceReflectance, vis_params, 'Surface Reflectance')

NameError: ignored

# Import section


In [ ]:
import numpy as np  # import numpy library as new name np
import os  # import os library
from os import path
import math
import matplotlib.pyplot as plt
import cv2
import gc
import datetime

print('libraries imported')

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# calculate num of months between two dates

In [ ]:
StMonth = 6 # define start month
StYear = 1986 # define start year
EnMonth = 1 # define end month
EnYear = 2010 # define end year
start_date = datetime.datetime(StYear, StMonth, 1) # create start date from data
end_date = datetime.datetime(EnYear, EnMonth, 1)# create end date from data
# calculate num of months between two dates
num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
print('Total Months=' + str(num_months))


In [ ]:
#NOTE:change patha acordingly
FilePath = "/content/drive/MyDrive/Database/GEE"

# create black and white mask

In [ ]:
# create black and white mask
Ratio = 30 # resize original image
#FilePath = 'D:\PROJECT\drought prediction\code\images\GEE' # image folder path in my pc
FileMask = 'maskIran.tif' # mask image name
MaskPath = os.path.join(FilePath, FileMask) # create full path
print(MaskPath)
Imask = cv2.imread(MaskPath, -1) # read image
Hm, Wm = Imask.shape # read size of image
Imask = cv2.resize(Imask, (round(Wm/Ratio), round(Hm/Ratio) )) # resize the image
Hm, Wm = Imask.shape # read size of image

print('Mask Size=' + str(Hm) + 'x' + str(Wm))
plt.imshow(Imask, cmap='gray')
plt.show()


# Create Stack of monthly TB from each month images

In [ ]:
# Create Stack of monthly TB from each month images
gc.collect()
import numpy as np

TBArray = np.zeros((num_months, Hm, Wm))# define blank array
print(TBArray.shape)
tmonth = StMonth
tyear = StYear
for i in range(0, num_months):
    if (tmonth < 10):
        SMonth = '0' + str(tmonth)
    else:
        SMonth = str(tmonth)

    #FName = 'TB' + str(tyear) + SMonth + '.tif' # create file name

    FName = str(tyear) + SMonth + '.tif' # create file name
    print(str(i) + ',' + FName)
    TBpath = os.path.join(FilePath, FName)# create full path
    tTB = cv2.imread(TBpath, -1)# read image
    tTB = cv2.resize(tTB, (Wm, Hm))# resize image

    # check for any Nan Value
    NanLoc = np.argwhere(np.isnan(tTB))# read location of NaN value
    NanLocLen = NanLoc.__len__()# calculate total nan value

    # remove nan values with 0
    for ii in range(0, NanLocLen):
        tLoc = NanLoc[ii] # read location of nan value
        tTB[tLoc[0]][tLoc[1]] = 0# replacee nan values with 0

    TBArray[i] = tTB# store to stack
    tmonth = tmonth + 1 # increment month
    if tmonth > 12: # if month >12
        tmonth = 1  # reset month to 1
        tyear = tyear + 1 # increment year

gc.collect()

print('TB Image stack generated')

# CREATE TCI VALUE STACK

In [ ]:


# check the data and process it
# import pickle
# pickle_out = open('TBStack.pickle','w', protocol=4)
# pickle.dump(TBArray,pickle_out)
# pickle_out.close()
# print('TB Data saved')
# Create Stack of monthly TB

##       CREATE TCI VALUE STACK
TCIm = np.zeros((num_months, Hm, Wm)) # create blank array
for i in range(0, Hm):
    for j in range(0, Wm):
        # TBlist = np.round(TBArray[i][j][0:num_months], 2)
        # TBlist = np.round(TBArray[0:num_months][i][j], 2)
        TBlist = np.round(TBArray[:, i, j], 2)  # read all 3rd elements
        LTBMax = np.round(np.max(TBlist), 2) # calculate max brightness temperature
        LTBMin = np.round(np.min(TBlist), 2) # calculate min brightness temperature
        print(str(i) + ',' + str(j))
        for k in range(0, num_months): # read all months
            # print(str(i) + ',' + str(j)+','+str(k))
            if (LTBMax == LTBMin): # if min, max same (in case of nan values)
                TCIm[k][i][j] = -10 # replace with -10
                print(str(i) + ',' + str(j) + ','+str(k)+',replaced')
            else:
                LTB = TBlist[k] # read that month brighness temperature
                tup = LTBMax - LTB
                tdw = LTBMax - LTBMin
                TCIval = np.round(100 * tup / tdw, 2) # apply formula for TCI
                TCIm[k][i][j] = TCIval # store TCI
            # print(str(i) + ',' + str(j) + ', TCI=' + str(TCIval))

print('TCI stack generated')


# CALCULATE MONTHLY AVERAGE AND LOG

In [ ]:

######################## CALCULATE MONTHLY AVERAGE AND LOG
from scipy import stats# import stats functions from scipy library
from scipy.stats import norm

MonthRange = 12
MonthlyAverageRange = num_months-(MonthRange-1)# total SPI value stack to be generated
TCIMonthlyAverage = np.zeros((MonthlyAverageRange, Hm, Wm))
TCIMonthlyAverageLog = np.zeros((MonthlyAverageRange, Hm, Wm))
for i in range(0, Hm):
    for j in range(0, Wm):
        print(str(i) + ',' + str(j))
        TCIlist = TCIm[:, i, j] # read TCI of that location
        for k in range (0, MonthlyAverageRange):
            tlist=TCIlist[k:k+12] # read next 12 month TCI
            tmean=np.round(np.mean(tlist),2) # calculate mean of TCI
            if (tmean != -10):
                TCIMonthlyAverage[k][i][j]=tmean # store mean of TCI
                TCIMonthlyAverageLog[k][i][j]=np.round(math.log(tmean),2) # store log of the (mean of TCI)
            else:
                TCIMonthlyAverage[k][i][j] = -10
                TCIMonthlyAverageLog[k][i][j] = -10

print('TCI monthly average, logs generated')

# Calculate SPI

In [ ]:

## Calculate SPI
SPI=np.zeros((MonthlyAverageRange, Hm, Wm))
ImgSPI = np.zeros((MonthlyAverageRange, Hm, Wm))
TCISumOfLog=np.zeros((Hm, Wm))
for i in range(0,Hm):
    for j in range(0, Wm):

        TCIAvgList = TCIMonthlyAverage[:, i, j] # read all monthly average of the location
        TCILogList = TCIMonthlyAverageLog[:, i, j] # read all monthly log of the location
        avgM = np.round(np.mean(TCIAvgList), 2) # calculate mean of (monthly average of location)
        if (avgM!=-10):
            sumL = np.sum(TCILogList) # sum of all logs
            A = np.round(np.log(avgM) - sumL / MonthlyAverageRange, 2) # calculate A
            alpha = np.round((1/(4*A))*(np.sqrt(1+(4*A/3))+1), 2)# calculate alpha
            beta = np.round( avgM / alpha) # calculate beta
            #print(str(i) + ',' + str(j))
            for k in range(0, MonthlyAverageRange):
                # calculate gamma cdf

                MonthAvg = TCIMonthlyAverage[k][i][j] # read that month TCI average of 12 month
                gcdf = stats.gamma.cdf(MonthAvg, alpha, scale=beta) # apply gamma cdf
                tspi = norm.ppf(gcdf)  # apply normalised inverse transformation
                tspi = np.round(tspi, 2)  # apply round off
                SPI[k][i][j] = tspi # put that SPI value in matrix
                print(str(k) + ',' + str(i) + ',' + str(j)+' (spi)='+str(tspi))

                # convert to integer bands
                if (tspi  >= 2.0):
                    ImgSPI[k][i][j] = 7
                elif (tspi >= 1.5):
                    ImgSPI[k][i][j] = 6
                elif (tspi >= 1.0):
                    ImgSPI[k][i][j] = 5
                elif (tspi >= -1.0):
                    ImgSPI[k][i][j] = 4
                elif (tspi >= -1.5):
                    ImgSPI[k][i][j] = 3
                elif (tspi >= -2.0):
                    ImgSPI[k][i][j] = 2
                else:
                    ImgSPI[k][i][j] = 1

        else:
            SPI[k][i][j] = -10
            ImgSPI[k][i][j] = 0

print('SPI stack generated')


# ploting


In [ ]:
## plot a random month SPI image
img= 36*ImgSPI[1] # our bands are within range 0 to  7, multiplying for proper view in 0 to 255 range
plt.imshow(img, cmap='YlGn')
plt.show()

In [ ]:

## plot a random coordinate SPI value graph over a time period
# plot graph of a physical locationn
from datetime import date, timedelta

dlist=[]
tmonth=5
tyear=1987
for i in range(MonthlyAverageRange): #MonthlyAverageRange
    if(tmonth<10):
        smonth = '0'+str(tmonth)
    else:
        smonth = str(tmonth)
    syear=str(tyear)
    dval=syear+'-'+smonth
    print(dval)
    dlist.append(dval)
    tmonth=tmonth+1
    if (tmonth>12):
        tmonth=1
        tyear=tyear+1

In [ ]:
ListSPI = SPI[:, 10, 10]
plt.plot(dlist, ListSPI)
plt.xticks(rotation='vertical')
plt.ylabel('SPI value')
plt.xlabel('Date')
plt.show()